# Resnet Multiple Image Inference Test

- original code : # https://pytorch.org/hub/pytorch_vision_resnet/
- modified by : JPark, Sep. 2022

In [8]:
# Make ONNX model
import torch
import torch.onnx
import os

def make_onnx_models(model_names = ['resnet18', 'resnet34']):
    for model_name in model_names:
        model = torch.hub.load('pytorch/vision:v0.10.0', model_name, pretrained=True)
        # model.eval()
        batch_size = 256
        x = torch.randn(batch_size, 3, 32, 32, requires_grad=True)
        fpath_model = "./checkpoint/" + model_name + ".onnx"

        torch.onnx.export(model,               # 실행될 모델
                          x,                         # 모델 입력값 (튜플 또는 여러 입력값들도 가능)
                          fpath_model,   # 모델 저장 경로 (파일 또는 파일과 유사한 객체 모두 가능)
                          export_params=True,        # 모델 파일 안에 학습된 모델 가중치를 저장할지의 여부
                          opset_version=10,          # 모델을 변환할 때 사용할 ONNX 버전
                          do_constant_folding=True,  # 최적하시 상수폴딩을 사용할지의 여부
                          input_names = ['input'],   # 모델의 입력값을 가리키는 이름
                          output_names = ['output'], # 모델의 출력값을 가리키는 이름
                          dynamic_axes={'input' : {0 : 'batch_size'},    # 가변적인 길이를 가진 차원
                                        'output' : {0 : 'batch_size'}})


model_names = [ 
    'resnet18',
    'resnet34',
    'resnet50',
    'resnet101',
    'resnet152', ]

make_onnx_models(model_names = model_names)

Using cache found in /Users/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/jpark/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/jpark/.cache/torch/hub/pytorch_vision_v0.10.0


### # Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

# ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt


In [28]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
import time


start = time.time()
for icnt in range(100):
    input_image = Image.open(filename)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model


    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)
        
    # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
    #print(output[0])
    
    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    #print(probabilities)

    # Read the categories
    with open("imagenet_classes.txt", "r") as f:
        categories = [s.strip() for s in f.readlines()]
    # Show top categories per image
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    for i in range(top5_prob.size(0)):
        pass
        #print(categories[top5_catid[i]], top5_prob[i].item())
        
end = time.time()
print(end - start)

35.063608169555664


## Test 실험결과

- 100회 반복, M1 MacbookPro14, CPU mode

```csv
model, time[sec]
resnet18, 9.702051162719727 
resnet34, 15.428618907928467
resnet50, 16.397982120513916
resnet101, 25.44774293899536
resnet152, 35.063608169555664
```